Python 3.7.x 에서 뉴스의 본문일부가 파싱이 제대로 되지 않는 문제가 있습니다. 이는 Python 3.6.x 에서 작성하였습니다.

이전의 네이버 영화 스크랩 코드에서 작성한 get_soup 과 normalize 함수를 utils.py 에 넣어뒀습니다. 이를 이용합니다.

In [1]:
import utils
from utils import get_soup
from utils import normalize

Python version = sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)
BeautifulSoup version = 4.7.1


아래 링크의 뉴스 기사를 가져와 제목과 본문을 가져옵니다. 이는 앞서 연습한 것처럼 inspect 를 이용하여 HTML 태그를 살펴보면 됩니다.

In [2]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=422&aid=0000370702'
soup = get_soup(url)

In [3]:
def parse_title(soup):
    return normalize(soup.select('h3[id=articleTitle]')[0].text)

parse_title(soup)

"강남경찰서 또 비위…피의자에게 금품수수 '입건'"

때로 br 태그를 이용하여 줄바꿈을 하는 경우들이 있는데, 이 정보를 살리기 위해 br 태그를 str 로 변환하였습니다.

In [4]:
from bs4 import BeautifulSoup

def parse_content(soup):
    html = str(soup.select("div[id=articleBodyContents]")[0])
    html = html.replace('<br/>', 'br/')
    content = BeautifulSoup(html, 'lxml').text
    content = '  '.join(normalize(s) for s in content.split('br/') if s.strip())
    return content

In [5]:
# parse_content(soup).split('  ')

위 코드를 정리하면 scrap_news 라는 함수를 만들 수 있습니다.

In [6]:
def scrap_news(url):
    json_obj = {'url': url}
    try:
        soup = get_soup(url)
        json_obj['title'] = parse_title(soup)
        json_obj['content'] = parse_content(soup)
        return json_obj
    except:
        return None

# scrap_news(url)

검색어를 입력하여 뉴스 기사들의 urls 을 가져오기 위한 함수도 만들어 봅니다. URL 에는 한글이 입력될 수 없습니다. 이들은 다른 글자로 인코딩이 되어야 합니다. URL 에 한글이 입력되어 있다면 이를 복사하여 다른 메모장에 붙여보시면 16진법으로 기술된 글자가 나옵니다. 한글이 16진법 숫자로 변환된 것인데, urllib 에서 이러한 기능을 제공합니다.

In [7]:
import urllib

def get_search_url(query, start_date, end_date):
    """
    query : str
        질의어
    start_date : str
        yyyy.mm.dd 형식 ex) 2019.04.18
    end_date : str
        yyyy.mm.dd 형식 ex) 2019.04.18
    """
    base = 'https://search.naver.com/search.naver?where=news&query={0}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={1}&de={2}'
    url = base.format(url_encode(query), start_date, end_date)
    return url

def url_encode(query, encoding='utf-8'):
    def encode_a_term(term):
        try:
            return urllib.parse.quote(term, encoding=encoding)
        except Exception as e:
            raise ValueError('Failed to encode query %s' % str(e))
    return '+'.join([encode_a_term(term) for term in query.split()])

In [8]:
url_encode('버닝썬')

'%EB%B2%84%EB%8B%9D%EC%8D%AC'

url_encode 함수를 이용하여 검색어, 시작일, 종료일이 입력되었을 때 뉴스 검색 결과를 가져오는 url 을 만듭니다.

In [9]:
base_url = get_search_url('버닝썬', '2019.04.18', '2019.04.19')
base_url

'https://search.naver.com/search.naver?where=news&query=%EB%B2%84%EB%8B%9D%EC%8D%AC&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=2019.04.18&de=2019.04.19'

뉴스가 몇 건 검색되었는지 숫자를 확인합니다.

In [10]:
import re

def get_article_num(url):
    try:
        soup = get_soup(url)
        if not soup:
            return 0
        header_text = soup.select('div[class=section_head] div[class^=title_desc] span')
        if not header_text:
            return 0
        header_text = header_text[0].text
        header_text = re.findall('[,\\d]+건', header_text)[0]
        header_text = re.sub(',', '', header_text) # Remove Comma
        num_articles = int(header_text[:-1])
        return num_articles

    except Exception as e:
        raise ValueError('Failed to get total number of articles %s' % str(e))

get_article_num(base_url)

660

특정 패턴이 있는 링크만 선택합니다.

In [11]:
def get_article_urls(base_url, page):

    url_patterns = ('a[href^="https://news.naver.com/main/read.nhn?"]',
            'a[href^="https://entertain.naver.com/main/read.nhn?"]',
            'a[href^="https://sports.news.naver.com/sports/index.nhn?"]',
            'a[href^="https://news.naver.com/sports/index.nhn?"]')

    urls_in_page = set()
    page_url = '{}&start={}&refresh_start=0'.format(base_url, 1 + 10*(page-1))
    soup = get_soup(page_url)
    if not soup:
        return urls_in_page
    try:
        article_blocks = soup.select('ul[class=type01]')[0]
        for pattern in url_patterns:
            article_urls = [link['href'] for link in article_blocks.select(pattern)]
            urls_in_page.update(article_urls)
    except Exception as e:
        raise ValueError('Failed to extract urls from page %s' % str(e))

    return urls_in_page

검색 결과로부터 뉴스 기사의 링크를 가져올 수 있습니다.

In [12]:
import time
import math

num_article = get_article_num(url)
num_article = 30 # debug
base_url = get_search_url('버닝썬', '2019.04.18', '2019.04.19')

urls = []
for page in range(1, math.ceil(num_article / 10) + 1):
    urls += get_article_urls(base_url, page)    
    print('getting urls from page = {}'.format(page))
    time.sleep(1)

len(urls)

getting urls from page = 1
getting urls from page = 2
getting urls from page = 3


39

댓글의 경우에는 크롬의 Network 를 이용하여 request url 을 가져올 수 있습니다. 여기에는 header 에 입력되는 user-agent 와 referer 의 값도 함께 기록되어 있습니다. 뉴스 기사의 url 이 referer 입니다. 이를 이용하여 API 주소를 가져옵니다.

![](news_comments.png)

In [13]:
import requests

referer_url = 'https://news.naver.com/main/read.nhn?m_view=1&mode=LSD&mid=shm&sid1=100&oid=008&aid=0004206900'
request_url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics&pool=cbox5&_callback=jQuery112406445509904006934_1555720412273&lang=ko&country=KR&objectId=news008%2C0004206900&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=2&refresh=false&sort=NEW&current=1700689291&prev=1700692531&includeAllStatus=true&_=1555720412275'
headers = {'Referer': referer_url}

r = requests.get(request_url, headers=headers)
text = r.text

response 의 시작과 끝에는 java script 관련 단어들이 포함되어 있습니다. 이를 제거한 뒤, JSON parsing 을 합니다. json.load 는 파일을 JSON 형식으로 읽는 함수이며, json.loads 는 str 을 JSON 형식으로 읽는 함수입니다.

In [14]:
import json

text = text[text.index('(')+1:-2]
response = json.loads(text)
# response

이를 정리하면 한 뉴스기사의 댓글을 가져오는 get_response 함수를 만들 수 있습니다.

In [15]:
def get_response(request_url, referer_url):
    r = requests.get(request_url, headers=headers)
    text = r.text
    text = text[text.index('(')+1:-2]
    return json.loads(text)

comments = get_response(request_url, referer_url)

In [16]:
comments.keys()

dict_keys(['success', 'code', 'message', 'lang', 'country', 'result', 'date'])